# Regression
In supervised learning, regression aims at predicting a continous numerical value using one or more independent features as input variables. It finds relationship between dependent variable (target) and independent variables (features). 

Different types of regression:
1. Simple linear regression: assumes there is linear correlation between dependent and independent variables. 
2. Multiple linear regression: extends simple linear regression with multiple independent variables as predictors.
3. Polynomial regression: used when there is non-linear relationship between dependent and independent variables.

This notebook covers only simple and multiple linear regression. 

## Load dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

We use **Student Performance** dataset from UCI ML repository. It contains data related to student achievement in secondary education, consisting of two subjects Mathematics (mat) and Portugese (por) language. The data has many features, including:
* Demographic (e.g., `age`, `sex`, `address`)
* Social and academic (e.g., `studytime`, `freetime`, `absences`)
* Grades: `G1` - 1st grade, `G2` - 2nd grade, and `G3` - final grade

For details of dataset information, including its features description, please visit the following website
https://archive.ics.uci.edu/dataset/320/student+performance

In [ ]:
studentDf = pd.read_csv('../dataset/student-mat.csv')
studentDf.head()

In [ ]:
studentDf.describe().T

In [ ]:
studentDf.isnull().sum()

## Visualize the data
### Univariate analysis

Plot distribution of numeric features

In [ ]:
fig = plt.figure(figsize=(16, 14))  
i = 0
for column in studentDf.select_dtypes(include=['number']):
    sub = fig.add_subplot(4, 4, i + 1)
    studentDf[column].plot(kind = 'hist')
    sub.set_xlabel(column)
    i = i + 1

### Correlation plot

In [ ]:
# correlation heat map
sns.heatmap(studentDf.select_dtypes(include=['number']).corr(), annot=False, cmap='viridis', 
            cbar_kws={'label': 'Correlation Coefficient', "shrink": .5})

## Prepare dataset
Define predictor and target variables. Split dataset into train and test set. 

In [ ]:
from sklearn.model_selection import train_test_split

X = studentDf['studytime'].to_numpy().reshape(-1, 1) # feature for predictor
y = studentDf['G3'].to_numpy().reshape(-1, 1) # target to predict

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

> Note that we had to perform `reshape` on the input data in order for the Linear Regression package to understand it correctly. Linear Regression expects a 2D-array as an input, where each row of the array corresponds to a vector of input features. In our case, since we have only one input - we need an array with shape N×1, where N is the dataset size.

## Train the linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression

# fit the ordinary linear regression (olr) model
olr = LinearRegression()
olr.fit(X_train, y_train)

olr

## Evaluate the model

In [ ]:
# predict using test dataset
y_pred = olr.predict(X_test)

# plot between true value and predicted value of G3 score
plt.scatter(y_test, y_pred)
plt.plot([0, 25], [0, 25], '--k')
plt.xlabel('True G3 score')
plt.ylabel('Predicted G3 score')

> The selected `studytime` feature does not serve as a good linear relationship predictor for `G3` score. As shown in the plot, regardless the true value of `G3` the predicted values barely change as the actual grade change. This suggest that `studytime` does not capture enough information to predict the `G3` accurately. See further evaluation below.

In [ ]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score

# evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MAPE: {mae/ y_test.mean() *100:.2f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

score = olr.score(X_train,y_train)
print(f'Model determination: {score:.4f}')

### 📊 Interpretation of Results

| Metric | Value | Meaning |
|--------|-------|---------|
| **MAE** | 3.63 | On average, the model's predictions are off by about 3.63 grade points. |
| **MAPE** | 35.22 | The average percentage error is 35.22%, which is quite high. |
| **MSE** | 21.85 | Squared error is high, indicating large deviations. |
| **RMSE** | 4.67 | Root mean squared error confirms the average error magnitude. |
| **R² Score** | 0.006 | The model explains only 0.6% of the variance in `G3`. Very poor fit. |
| **Model Determination** | 0.0109 | Same as R², confirms very weak predictive power. |


---

### 🧠 What This Means?

- Align with the prior plot, **`studytime` alone is a weak predictor** of final grade (`G3`).
- The **high error metrics** (MAE, RMSE, MAPE) suggest the model is not reliable for prediction.
- The R² score is close to zero, meaning the model barely explains any variation in the target.

---

### 🔍 More about R²

**R²** and **model determination** refer to the same metric, reflecting how well the model explains the variance in the target variable (`G3`). 

**Formula**:  
  $$ R^2 = 1 - \frac{\text{SS}_{\text{res}}}{\text{SS}_{\text{tot}}} $$
where:
  - $ \text{SS}_{\text{res}} $ = sum of squared residuals (errors)
  - $ \text{SS}_{\text{tot}} $ = total sum of squares (variance of the target)

**Interpretation**:
  - R² = 1 → perfect prediction
  - R² = 0 → model does no better than the mean
  - R² < 0 → model is worse than just predicting the mean

As you may notice, the score of **R²** and **model determination** are different. This is because they used different dataset. **R²** was applied for **test data**, while **model determination** was calculated against **training data**. 

- Calculate both are essential for evaluating wheather the model overfits. **R²** score on training will be higher than test in such case.
- **R²** score on test dataset indicates how well the model generalizes to unseen data.

---



## Extract model insights

In [ ]:
print(f"Intercept: {olr.intercept_[0]:.2f}")
print(f"Coefficient: {olr.coef_[0][0]:.2f}")

> **Intercept** is the baseline value of the dependent variable when there is no contribution from the predictors.
> The **coefficient is positive**, which makes sense: more study time tends to correlate with better grades, but the effect is small.
> From the both values we can formulate linear model as follows:
> $$ y = 9.28 + 0.57x_1$$
> where $y$ denotes `G3` and $x_1$ refers to `studytime`

## Train with another predictor

In [ ]:
X = studentDf['G1'].to_numpy().reshape(-1, 1) # feature for predictor
y = studentDf['G3'].to_numpy().reshape(-1, 1) # target to predict

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# fit the ordinary linear regression (olr) model
olr = LinearRegression()
olr.fit(X_train, y_train)

olr

## Evaluate the model with new predictor

In [ ]:
# predict using test dataset
y_pred = olr.predict(X_test)

# plot between true value and predicted value of G3 score
plt.scatter(y_test, y_pred)
plt.plot([0, 25], [0, 25], '--k')
plt.xlabel('True G3 score')
plt.ylabel('Predicted G3 score')

In [ ]:
# evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MAPE: {mae/ y_test.mean() *100:.2f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

score = olr.score(X_train,y_train)
print(f'Model determination: {score:.4f}')

### 📊 Interpretation of Results

The model now improved with better performance.

| Metric | Value | Meaning |
|--------|-------|---------|
| **MAE** | 1.83 | On average, the model's predictions are off by about 1.83 grade points. |
| **MAPE** | 17.75 | The average percentage error is 17.75%, which is reasonably acceptable. |
| **MSE** | 6.66 | Squared error is approximately three times smaller than previous result. |
| **RMSE** | 2.58 | Root mean squared error confirms the average error magnitude. |
| **R² Score** | 0.6971 | The model explains 69% of the variance in `G3`. Such a good fit. |
| **Model Determination** | 0.6165 | Same as R², confirms strong predictive power againts training data. |

The small gap between **R²** and **Model Determination** indicate the model did not suffer from overfitting. 


## New coefficient

In [ ]:
print(f"Intercept: {olr.intercept_[0]:.2f}")
print(f"Coefficient: {olr.coef_[0][0]:.2f}")

> The baseline **intercept** value decreases significantly. Based on the value, it means that the model would predict `G3` to be -1.81 when the student `G1` score is 0.
> 
> The **coefficient** tends to be a strong positive relationship with better `G3` grades. If the `G1` point increases 1 point, the `G3` score is predicted to increase by 1.11 points
> 
> From the both values we can formulate linear model as follows:
> $$ y = -1.81 + 1.11x_1$$
> where $y$ denotes `G3` and $x_1$ refers to `G1`

# Multiple Regression 

Let's now try to use all numeric features as predictor and see if they can improve the model's performance.

In [ ]:
X = studentDf.select_dtypes(include=['number']).drop('G3', axis=1) # select all numeric features except G3
y = studentDf['G3'].to_numpy().reshape(-1, 1) # target to predict

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# fit the ordinary linear regression (olr) model
multiple_reg_model = LinearRegression()
multiple_reg_model.fit(X_train, y_train)

# predict using test dataset
y_pred = multiple_reg_model.predict(X_test)

# evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MAPE: {mae/ y_test.mean() *100:.2f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

score = multiple_reg_model.score(X_train,y_train)
print(f'Model determination: {score:.4f}')

> Compared to the prior model, the new model has improved the performance by approximately **15%** based on the R² score when considering more features. 

### Extract new model insights

In [ ]:
print(f"Intercept: {multiple_reg_model.intercept_[0]:.2f}")

coeff_df = pd.DataFrame({"Feature": X.columns, "Coefficient": multiple_reg_model.coef_[0]})
coeff_df

In [ ]:
# Sort dataframe by coefficients.
coeff_df_sorted = coeff_df.sort_values(by="Coefficient", ascending=False)

# Create plot.
plt.figure(figsize=(8,6))
plt.barh(coeff_df_sorted["Feature"], coeff_df_sorted["Coefficient"], color="blue")
plt.xlabel("Coefficient Value")
plt.ylabel("Feature")
plt.title("Feature Importance (Linear Regression Coefficients)")
plt.show()

In [ ]:
# Compute residuals.
residuals = y_test - y_pred

# Create plots.
plt.figure(figsize=(12,5))

# Plot 1: Residuals Distribution.
plt.subplot(1,2,1)
sns.histplot(residuals, bins=30, kde=True, color="blue")
plt.axvline(x=0, color='red', linestyle='--')
plt.title("Residuals Distribution")
plt.xlabel("Residuals (y_actual - y_predicted)")
plt.ylabel("Frequency")

############# ------------------------- #############

# Plot 2: Regression Fit (Actual vs Predicted).
plt.subplot(1,2,2)
plt.scatter(y_test, y_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')  # Perfect fit line
plt.title("Regression Fit: Actual vs Predicted")
plt.xlabel("Actual G3")
plt.ylabel("Predicted G3")


# Show plots.
plt.tight_layout()
plt.show()

### 📊 Interpretation 

The **residuals distribution** indicate that errors are randomly distributed. 

- The model does not under- or over-fitting
- When the residuals follow a normal distribution, the model fits well
- If there is **skewness** or too heavy tails, it may suggest non-linear relationship not captured by the model

The **regression fit** compares actual vs. predicted values, with the red dashed line representing a perfect fit. 

- If points closely follow the line, predictions are accurate
- but if points scatte deviating the line, the relationship may not be truly linear, indicating limited predictive power (see the previous result when using only `studytime` as predictor)